用于转化图像类型和通道

In [ ]:
import cv2
import numpy

img_path = "data//train//Image1_1.jpg"
img = cv2.imread(img_path)
img_RGB = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
img_array = numpy.array(img_RGB)

print(img_array.shape) 
writer.add_image("test", img_array, 3, dataformats='HWC')
writer.close()

用于creat Dataset

In [36]:
from torchvision import transforms
from torch.utils.tensorboard import SummaryWriter
from PIL import Image
import os
from torch.utils.data import Dataset, DataLoader
import numpy as np

class MyData(Dataset):

    def __init__(self, root_dir,  label_dir, dataset_type, transform):
        self.root_dir = root_dir
        self.dataset_type = dataset_type 

        self.image_path = os.path.join(self.root_dir, self.dataset_type)

        self.image_list = os.listdir(self.image_path)  # 因为label 和 Image文件名相同，进行一样的排序，可以保证取出的数据和label是一一对应的
        # self.label_list = os.listdir(self.label_path)

        #cut to patches
        self.transform = transform
        
    def __getitem__(self, idx):
        img_name = self.image_list[idx]
        # label_name = self.label_list[idx]
        img_item_path = os.path.join(self.root_dir, self.dataset_type, img_name)

        # label_item_path = os.path.join(self.root_dir, self.label_dir, label_name)
        img = Image.open(img_item_path)

        # img = np.array(img)
        img = self.transform(img)
        
        sample = {'img': img}
        return sample

    def __len__(self):
        return len(self.image_list)



if __name__ == '__main__':
    writer = SummaryWriter("attempt_1_logs")
    
    transform = transforms.Compose([transforms.Grayscale(num_output_channels=1), transforms.ToTensor()])        ##opt:transforms.resize(32)
    root_dir = "data"
    dataset_type = "train"
    label = "RGB"
    RGB_dataset = MyData(root_dir, label, dataset_type, transform)

    img1 = RGB_dataset[0]['img']
    writer.add_image('RGB', img1)         #return is a dict

    # images = crop_image(img1, 32, 32)
    # train_loader = DataLoader(RGB_dataset, batch_size = patch_num)      #可用先用来看看分割的结果

    # step = 0
    # for data in train_loader:
    #     writer.add_images('RGBs', img1, step)         #return is a dict
    #     step = step + 1

    # writer.close()

In [ ]:
from torchvision import transforms
import torch
from torch.utils.tensorboard import SummaryWriter
from PIL import Image
import os
from torch.utils.data import Dataset, DataLoader
import numpy as np
import cv2
def crop_image(img, height, width):
    """cut the image into patches- input/output: tensor """

    H = img.shape[1]
    W = img.shape[2]
    patch_num = min( H // height,  W // width)**2        #采取最大的正方形面积

    imgs = img[:, 0:height*1, 0:width*1]
    imgs = imgs.unsqueeze(0)
    for i in range(2, patch_num + 1):
        for j in range(1, patch_num + 1):
            img_current = img[:, height*(i-1):height*i, height*(j-1):width*j]
            img_current = img_current.unsqueeze(0)
            imgs = torch.cat([imgs, img_current], dim=0)
            
    return imgs

if __name__ == '__main__':
    
    img1 = RGB_dataset[0]['img']
    images = crop_image(img1, 32, 32)

    writer = SummaryWriter("attempt_1_logs")
    step = 0
    for i in range(images.shape[0]):
        writer.add_image('RGBs', images[i], step)         #return is a dict
        step = step + 1
